In [0]:
import requests
import urllib3
import json
import csv
import time
from datetime import datetime, timedelta, date
from zipfile import ZipFile
import io
import os

In [0]:
urllib3.disable_warnings()

In [0]:
def get_key(client_id, client_secret):
    key_request = requests.post('https://performance.ozon.ru:443/api/client/token',
                                headers={
                                    "Content-Type": "application/json",
                                    "Accept": "application/json"
                                },
                                json={
                                    "client_id": client_id,
                                    "client_secret": client_secret,
                                    "grant_type": "client_credentials"
                                }
                                )
    return key_request.json()['access_token']


def get_compaign_list():
    compaign_request = requests.get('https://performance.ozon.ru:443/api/client/campaign',
                                    headers=headers
                                    )

    campaign = compaign_request.json()
    n = 10
    campaign_id_list_for_campaigns = [i["id"] for i in campaign["list"] if i["state"] == "CAMPAIGN_STATE_RUNNING" or i["state"] == "CAMPAIGN_STATE_INACTIVE"]
    campaign_id_list_for_campaigns_divided = [campaign_id_list_for_campaigns[i * n:(i + 1) * n] for i in range((len(campaign_id_list_for_campaigns) + n - 1) // n)]
    campaign_id_list_for_phrases = [i["id"] for i in campaign["list"] if i["state"] == "CAMPAIGN_STATE_RUNNING" and i["productCampaignMode"] == "PRODUCT_CAMPAIGN_MODE_MANUAL" or i["state"] == "CAMPAIGN_STATE_INACTIVE" and i["productCampaignMode"] == "PRODUCT_CAMPAIGN_MODE_MANUAL"]
    campaign_id_list_for_phrases_divided = [campaign_id_list_for_phrases[i * n:(i + 1) * n] for i in range((len(campaign_id_list_for_phrases) + n - 1) // n)]
    campaign_id_list_for_attribution = [i["id"] for i in campaign["list"] if (i["state"] == "CAMPAIGN_STATE_RUNNING" or i["state"] == "CAMPAIGN_STATE_INACTIVE") and (i["advObjectType"] == "BANNER" or i["advObjectType"] == "BRAND_SHELF")]
    campaign_id_list_for_attribution_divided = [campaign_id_list_for_attribution[i * n:(i + 1) * n] for i in range((len(campaign_id_list_for_attribution) + n - 1) // n)]
    return {"campaign_id_list_for_campaigns": campaign_id_list_for_campaigns_divided, 
            "campaign_id_list_for_phrases": campaign_id_list_for_phrases_divided,
            "campaign_id_list_for_attribution": campaign_id_list_for_attribution_divided
            }



def get_report_id(url, campaign_id):
    statistisc_request = requests.post(url,
                                       headers=headers,
                                       json={
                                           "campaigns": campaign_id,
                                           "dateFrom": f"{datetime.strftime(date_from, '%Y-%m-%d')}",
                                           "dateTo": f"{datetime.strftime(date_to, '%Y-%m-%d')}",
                                           "groupBy": "DATE"
                                       }
                                       )
    return statistisc_request.json()["UUID"]


def get_report_status(report_id):
    status_request = requests.get(f'https://performance.ozon.ru:443/api/client/statistics/{report_id}',
                                  headers=headers
                                  )
    return status_request.json()["state"]


def get_report_files(report_id):
    files_request = requests.get('https://performance.ozon.ru:443/api/client/statistics/report',
                                 headers=headers,
                                 params={
                                     "UUID": report_id
                                 }
                                 )
    return files_request.content

In [0]:
lk_dict = {
    "mars_trade": {
        "client_id": "2946364-1665124077199@advertising.performance.ozon.ru",
        "client_secret": "8wxQ0XZF70RmEOynZQi7HdRhNgqPgW1XnMrHTZQ8uldAlnEDCews2WBOPJyF8CjHD7W7s5iEj1RyWpSVeA"
    },
    "korkunov_trade": {
        "client_id": "4045534-1676053545827@advertising.performance.ozon.ru",
        "client_secret": "gKii-4dZlVlWYYTcsG9aVkt8PasCZuN6evt9EG9KYE-g8OKsf4A4R-D-UA_p_T9Uy7n2ID0PLIPYW2mxjA"
    }
}

date_from = date.today() - timedelta(days=7)
date_to = date.today()

unix_time_now = str(int(datetime.today().timestamp()))
year_now = str(datetime.today().year)
month_now = str(datetime.today().month).zfill(2)
day_now = str(datetime.today().day).zfill(2)

In [0]:
required_reports = {
    "campaign_statistics": 'https://performance.ozon.ru:443/api/client/statistics',
    "phrases_statistics": 'https://performance.ozon.ru:443/api/client/statistics/phrases',
    "attribution_statistics": 'https://performance.ozon.ru:443/api/client/statistics/attribution'
    }

In [0]:
ingest_layer_destination = f"/dbfs/mnt/DEV/RAW/FILES/RUSSIA_MW_DATAROOM/ingest/Ecom/ozon_api/{year_now}/{month_now}/{day_now}"

In [0]:
waiting = 60
retry = 50

##### campaign_statistics 

In [0]:
for lk in lk_dict:

  print(f"Proccess for account {lk} starting...")  
  access_token = get_key(lk_dict[lk]["client_id"], lk_dict[lk]["client_secret"])
  print("Access token recevied.")

  headers = {
      "Content-Type": "application/json",
      "Accept": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  campaign_id_list = get_compaign_list()

  for each_10 in campaign_id_list["campaign_id_list_for_campaigns"]:
    print(f'Process for campaignes {each_10} starting ...')
    uuid = get_report_id(required_reports["campaign_statistics"], each_10)
    print(f'UUID is {uuid}')
    attempt = retry
    time.sleep(waiting)
    report_status = get_report_status(uuid)
    while report_status != 'OK':
        attempt -= 1
        print(f'The reports for {each_10} are not ready yet, waiting for {waiting} sec to retry ...')
        time.sleep(waiting)
        report_status = get_report_status(uuid)
        if attempt == 0:
            print('Attempt limit has been reached ...')
            break
    if len(each_10) > 1:
        zip_archive = ZipFile(io.BytesIO(get_report_files(uuid)))
        zip_archive.extractall(f"{ingest_layer_destination}/campaign_statistics/{lk}")
        print(f'Reports for campaigns {each_10} have been saved.')
        time.sleep(waiting)
    else:
        os.makedirs(f"{ingest_layer_destination}/attribution_statistics/{lk}", exist_ok=True)
        with open(f"{ingest_layer_destination}/campaign_statistics/{lk}/{str(*each_10)}_{datetime.strftime(date_from, '%d.%m.%Y')}-{datetime.strftime(date_to, '%d.%m.%Y')}.csv", "wb") as file:
            file.write(get_report_files(uuid))
            print(f'Report for campaign {each_10} has been saved.')
            time.sleep(waiting)
  print(f"Proccess for account {lk} completed!")

Proccess for account mars_trade starting...
Access token recevied.
Process for campaignes ['3448662', '3446798', '3444321', '3444273', '3444193', '3444135', '3415819', '3346805', '3346785', '2983708'] starting ...
UUID is ef71f33a-0e3b-4f53-8202-173a91fab905
Reports for campaigns ['3448662', '3446798', '3444321', '3444273', '3444193', '3444135', '3415819', '3346805', '3346785', '2983708'] have been saved.
Process for campaignes ['2953196', '2953144', '2953119', '2953064', '2953022', '2519490', '2501769', '2454235', '2454215', '2352410'] starting ...
UUID is 11d2c47d-3290-4faa-9fe7-d4eeab7be0e5
Reports for campaigns ['2953196', '2953144', '2953119', '2953064', '2953022', '2519490', '2501769', '2454235', '2454215', '2352410'] have been saved.
Process for campaignes ['2339215', '2314370', '1857122', '1658502', '1658497', '1634100', '1406081', '1254992', '1254989', '1252110'] starting ...
UUID is 5faf20a3-479a-476a-94c6-97940b8961dd
The reports for ['2339215', '2314370', '1857122', '165850

##### phrases_statistics

In [0]:
for lk in lk_dict:

  print(f"Proccess for account {lk} starting...")  
  access_token = get_key(lk_dict[lk]["client_id"], lk_dict[lk]["client_secret"])
  print("Access token recevied.")

  headers = {
      "Content-Type": "application/json",
      "Accept": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  campaign_id_list = get_compaign_list()

  for each_10 in campaign_id_list["campaign_id_list_for_phrases"]:
    print(f'Process for campaignes {each_10} starting ...')
    uuid = get_report_id(required_reports["phrases_statistics"], each_10)
    print(f'UUID is {uuid}')
    attempt = retry
    time.sleep(waiting)
    report_status = get_report_status(uuid)
    while report_status != 'OK':
        attempt -= 1
        print(f'The reports for {each_10} are not ready yet, waiting for {waiting} sec to retry ...')
        time.sleep(waiting)
        report_status = get_report_status(uuid)
        if attempt == 0:
            print('Attempt limit has been reached ...')
            break
    if len(each_10) > 1:
        zip_archive = ZipFile(io.BytesIO(get_report_files(uuid)))
        zip_archive.extractall(f"{ingest_layer_destination}/phrases_statistics/{lk}")
        print(f'Reports for campaigns {each_10} have been saved.')
        time.sleep(waiting)
    else:
        os.makedirs(f"{ingest_layer_destination}/attribution_statistics/{lk}", exist_ok=True)
        with open(f"{ingest_layer_destination}/phrases_statistics/{lk}/{str(*each_10)}_{datetime.strftime(date_from, '%d.%m.%Y')}-{datetime.strftime(date_to, '%d.%m.%Y')}.csv", "wb") as file:
            file.write(get_report_files(uuid))
            print(f'Report for campaign {each_10} has been saved.')
            time.sleep(waiting)
  print(f"Proccess for account {lk} completed!")

Proccess for account mars_trade starting...
Access token recevied.
Process for campaignes ['3448662', '3446798', '3444321', '3444273', '3444193', '3444135', '3415819', '3346805', '3346785', '2983708'] starting ...
UUID is b3c8bb3b-8568-4d69-8710-54738126751d
Reports for campaigns ['3448662', '3446798', '3444321', '3444273', '3444193', '3444135', '3415819', '3346805', '3346785', '2983708'] have been saved.
Process for campaignes ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '2454235', '2352410', '2314370', '1857122'] starting ...
UUID is 6b055674-2771-4a68-877c-90df9f250408
Reports for campaigns ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '2454235', '2352410', '2314370', '1857122'] have been saved.
Process for campaignes ['1634100', '1406081', '1254992', '1254989', '1252110', '1220328', '1144843', '979823', '979820', '979817'] starting ...
UUID is 7efe4f82-257a-4dcd-92ad-e6f5fe5dd642
Reports for campaigns ['1634100', '1406081', '1254992', '125

#####attribution_statistics

In [0]:
for lk in lk_dict:

  print(f"Proccess for account {lk} starting...")  
  access_token = get_key(lk_dict[lk]["client_id"], lk_dict[lk]["client_secret"])
  print("Access token recevied.")

  headers = {
      "Content-Type": "application/json",
      "Accept": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  campaign_id_list = get_compaign_list()

  for each_10 in campaign_id_list["campaign_id_list_for_attribution"]:
    print(f'Process for campaignes {each_10} starting ...')
    uuid = get_report_id(required_reports["attribution_statistics"], each_10)
    print(f'UUID is {uuid}')
    attempt = retry
    time.sleep(waiting)
    report_status = get_report_status(uuid)
    while report_status != 'OK':
        attempt -= 1
        print(f'The reports for {each_10} are not ready yet, waiting for {waiting} sec to retry ...')
        time.sleep(waiting)
        report_status = get_report_status(uuid)
        if attempt == 0:
            print('Attempt limit has been reached ...')
            break
    if len(each_10) > 1:
        zip_archive = ZipFile(io.BytesIO(get_report_files(uuid)))
        zip_archive.extractall(f"{ingest_layer_destination}/attribution_statistics/{lk}")
        print(f'Reports for campaigns {each_10} have been saved.')
        time.sleep(waiting)
    else:
        os.makedirs(f"{ingest_layer_destination}/attribution_statistics/{lk}", exist_ok=True)
        with open(f"{ingest_layer_destination}/attribution_statistics/{lk}/{str(*each_10)}_{datetime.strftime(date_from, '%d.%m.%Y')}-{datetime.strftime(date_to, '%d.%m.%Y')}.csv", "wb") as file:
            file.write(get_report_files(uuid))
            print(f'Report for campaign {each_10} has been saved.')
            time.sleep(waiting)
  print(f"Proccess for account {lk} completed!")

Proccess for account mars_trade starting...
Access token recevied.
Process for campaignes ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '1406081', '896120', '896089', '590058'] starting ...
UUID is 5d752ef8-54b3-4f76-a7da-607047ad3cd9
The reports for ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '1406081', '896120', '896089', '590058'] are not ready yet, waiting for 60 sec to retry ...
The reports for ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '1406081', '896120', '896089', '590058'] are not ready yet, waiting for 60 sec to retry ...
The reports for ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '1406081', '896120', '896089', '590058'] are not ready yet, waiting for 60 sec to retry ...
The reports for ['2953196', '2953144', '2953119', '2953064', '2953022', '2501769', '1406081', '896120', '896089', '590058'] are not ready yet, waiting for 60 sec to retry ...
Reports for campaigns ['2953196', '2953144',